In [226]:
import yfinance as yf
import pandas as pd
import numpy as np

In [227]:
list=["HDFCBANK.NS", "ICICIBANK.NS", "AXISBANK.NS", "KOTAKBANK.NS", "SBIN.NS", "INDUSINDBK.NS", "BANKBARODA.NS",
     "FEDERALBNK.NS", "IDFCFIRSTB.NS", "PNB.NS"]

In [228]:
ticker1 = "AXISBANK.NS"

# Download 3 months of historical data using the daily time frame
data1 = yf.download(ticker1, start='2020-01-01', end='2022-01-01')[['Adj Close']]
len(data1)

arr = [[0 for i in range(len(data1))] for j in range(len(list))]

[*********************100%***********************]  1 of 1 completed


In [229]:
for i in range(len(list)):
    ticker=list[i]
    data=yf.download(ticker, start='2020-01-01', end='2022-01-01')[['Adj Close']]
    df_=pd.DataFrame(data)
    arr[i]=df_.to_numpy().reshape(1,-1)[0]
#     print(len(arr[i]))
    
# print(arr)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [230]:
ret = [[0 for i in range(len(data1)-1)] for j in range(len(list))]
for i in range(len(list)):
    for j in range(1,len(data1)):
        ret[i][j-1]=arr[i][j]/arr[i][j-1]-1

In [231]:
correlation_matrix = np.corrcoef(ret, rowvar=True)

# len(correlation_matrix)

np.fill_diagonal(correlation_matrix, -1)

max_value = np.max(correlation_matrix)
max_indices = np.unravel_index(np.argmax(correlation_matrix), correlation_matrix.shape)

print("Maximum value excluding 1:", max_value)
print("Indices of the maximum value:", list[max_indices[0]], list[max_indices[1]])

Maximum value excluding 1: 0.7983076687568065
Indices of the maximum value: ICICIBANK.NS AXISBANK.NS


In [232]:
s1=max_indices[0]
s2=max_indices[1]

ratio=arr[s1]/arr[s2]
m=np.mean(ratio)
s=np.std(ratio)

mean1=np.mean(arr[s1])
mean2=np.mean(arr[s2])

In [233]:
ticker1 = list[s1]

# Download 3 months of historical data using the daily time frame
data1 = yf.download(ticker1, start='2022-01-01', end='2023-01-01')[['Adj Close']]
df1=pd.DataFrame(data1)

ticker2 = list[s2]

# Download 3 months of historical data using the daily time frame
data2 = yf.download(ticker2, start='2022-01-01', end='2023-01-01')[['Adj Close']]
df2=pd.DataFrame(data2)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [234]:
from scipy.stats import norm
df_ratio.drop(df_ratio.columns, axis=1, inplace=True)
df_ratio['1st']=df1
df_ratio['2nd']=df2
df_ratio['ratio']=df1/df2
df_ratio['cdf']=norm.cdf(df_ratio['ratio'], loc=m, scale=s)
df_ratio['action']=0
df_ratio['state']='chill'

In [235]:
df_ratio

,1st,2nd,ratio,cdf,action,state
Date,,,,,,
2022-01-03,760.138428,693.864136,1.095515,0.997340,0,chill
2022-01-04,768.239807,706.618530,1.087206,0.996437,0,chill
2022-01-05,783.349121,724.305115,1.081518,0.995668,0,chill
2022-01-06,780.367004,727.693054,1.072385,0.994122,0,chill
2022-01-07,788.518127,727.991943,1.083141,0.995902,0,chill
...,...,...,...,...,...,...
2022-12-16,902.000000,933.180176,0.966587,0.902364,0,chill
2022-12-19,906.450012,944.257446,0.959961,0.888476,0,chill
2022-12-20,909.700012,947.800171,0.959801,0.888126,0,chill


In [236]:
for i in range(len(df_ratio.index)):
    idx=df_ratio.index[i]
    x=df_ratio.loc[idx, 'cdf']
    state=df_ratio.loc[idx, 'state']
    if(idx==df_ratio.index[-1]):
        break
    nxt=df_ratio.index[i+1]
    
    if(state=='chill'):
        if(x>0.003 and x<0.025):
            df_ratio.loc[idx, 'action']='buy'
            df_ratio.loc[nxt, 'state']='after buy'
        elif(x<0.997 and x>0.975):
            df_ratio.loc[idx, 'action']='sell'
            df_ratio.loc[nxt, 'state']='after sell'
        else:
            df_ratio.loc[idx, 'action']='wait'
            df_ratio.loc[nxt, 'state']='chill'
    
    if(state=='after buy'):
        if(x<=0.003 or x>=0.025):
            df_ratio.loc[idx, 'action']='sell'
            df_ratio.loc[nxt, 'state']='chill'
        else:
            df_ratio.loc[idx, 'action']='wait'
            df_ratio.loc[nxt, 'state']='after buy'
            
    if(state=='after sell'):
        if(x<=0.975 or x>=0.997):
            df_ratio.loc[idx, 'action']='buy'
            df_ratio.loc[nxt, 'state']='chill'
        else:
            df_ratio.loc[idx, 'action']='wait'
            df_ratio.loc[nxt, 'state']='after sell'

In [237]:
pd.reset_option('display.max_rows')
df_ratio

,1st,2nd,ratio,cdf,action,state
Date,,,,,,
2022-01-03,760.138428,693.864136,1.095515,0.997340,wait,chill
2022-01-04,768.239807,706.618530,1.087206,0.996437,sell,chill
2022-01-05,783.349121,724.305115,1.081518,0.995668,wait,after sell
2022-01-06,780.367004,727.693054,1.072385,0.994122,wait,after sell
2022-01-07,788.518127,727.991943,1.083141,0.995902,wait,after sell
...,...,...,...,...,...,...
2022-12-16,902.000000,933.180176,0.966587,0.902364,wait,chill
2022-12-19,906.450012,944.257446,0.959961,0.888476,wait,chill
2022-12-20,909.700012,947.800171,0.959801,0.888126,wait,chill


In [238]:
df_trunk=df_ratio
for i in range(len(df_ratio)-1, -1, -1):
    if(df_trunk.loc[df_ratio.index[i], 'action']=='wait' and df_trunk.loc[df_ratio.index[i], 'state']=='chill'):
        break
    else:
        df_trunk.drop(df_ratio.index[i], inplace=True)
        
df_trunk

,1st,2nd,ratio,cdf,action,state
Date,,,,,,
2022-01-03,760.138428,693.864136,1.095515,0.997340,wait,chill
2022-01-04,768.239807,706.618530,1.087206,0.996437,sell,chill
2022-01-05,783.349121,724.305115,1.081518,0.995668,wait,after sell
2022-01-06,780.367004,727.693054,1.072385,0.994122,wait,after sell
2022-01-07,788.518127,727.991943,1.083141,0.995902,wait,after sell
...,...,...,...,...,...,...
2022-12-15,908.799988,936.223938,0.970708,0.910336,wait,chill
2022-12-16,902.000000,933.180176,0.966587,0.902364,wait,chill
2022-12-19,906.450012,944.257446,0.959961,0.888476,wait,chill


In [239]:
bought_amount=0
sold_amount=0
for i in df_trunk.index:
    if(df_trunk.loc[i, 'action']=='buy'):
        bought_amount+=df_trunk.loc[i, '1st']
        sold_amount+=df_trunk.loc[i, '2nd']
        
    elif(df_trunk.loc[i, 'action']=='sell'):
        bought_amount+=df_trunk.loc[i, '2nd']
        sold_amount+=df_trunk.loc[i, '1st']
        
print(bought_amount)word
print(sold_amount)

18797.52423095703
18841.830444335938
